# Machine Learning Tutorial Python - 16: Hyper parameter Tuning (GridSearchCV)

- choosing best model


In [139]:
from sklearn.datasets import load_iris

iris = load_iris()
dir(iris)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [140]:
import pandas as pd

df = pd.DataFrame(iris.data, columns=iris.feature_names)
df["flower"] = iris.target
df.flower = df.flower.apply(lambda x: iris.target_names[x])
df[47:52]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor


In [141]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.3
)

In [142]:
from sklearn import svm  # assume is best model

model = svm.SVC(kernel="rbf", C=30, gamma="auto")
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.9777777777777777

In [143]:
# use cross validation to fold data, get avergae score,  and get better view of model accuracy
from sklearn.model_selection import cross_val_score

print(
    cross_val_score(
        svm.SVC(kernel="linear", C=10, gamma="auto"), iris.data, iris.target, cv=5
    )
)
print(
    cross_val_score(
        svm.SVC(kernel="rbf", C=10, gamma="auto"), iris.data, iris.target, cv=5
    )
)
print(
    cross_val_score(
        svm.SVC(kernel="rbf", C=20, gamma="auto"), iris.data, iris.target, cv=5
    )
)

[1.         1.         0.9        0.96666667 1.        ]
[0.96666667 1.         0.96666667 0.96666667 1.        ]
[0.96666667 1.         0.9        0.96666667 1.        ]


In [144]:
import numpy as np

kernels = ["rbf", "linear"]
C = [1, 10, 20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(
            svm.SVC(kernel=kval, C=cval, gamma="auto"), iris.data, iris.target, cv=5
        )
        avg_scores[kval + "_" + str(cval)] = np.average(cv_scores)

avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

In [145]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(
    svm.SVC(gamma="auto"),
    {
        "C": [1, 10, 20],
        # 'C': range(10),
        "kernel": ["rbf", "linear"],
    },
    cv=5,
    return_train_score=False,
)
clf.fit(iris.data, iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00174236, 0.00079212, 0.00092411, 0.00056419, 0.00064988,
        0.00074649]),
 'std_fit_time': array([1.03508157e-04, 4.61787076e-04, 6.23287856e-04, 6.70912762e-05,
        7.01622414e-05, 2.90008737e-04]),
 'mean_score_time': array([0.00272212, 0.00044165, 0.00046959, 0.00037789, 0.00039167,
        0.00050993]),
 'std_score_time': array([1.83659699e-03, 9.61742367e-05, 1.77851989e-04, 4.29277932e-05,
        1.70541128e-05, 2.66644226e-04]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value=999999),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20

In [146]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001742,0.000104,0.002722,0.001837,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000792,0.000462,0.000442,0.000096,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.000924,0.000623,0.000470,0.000178,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.000564,0.000067,0.000378,0.000043,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.000650,0.000070,0.000392,0.000017,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.000746,0.000290,0.000510,0.000267,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [147]:
df[["param_C", "param_kernel", "mean_test_score"]]
# hypertuned

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [148]:
clf.best_score_

0.9800000000000001

In [149]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [150]:
from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(
    svm.SVC(gamma="auto"),
    {"C": [1, 10, 20], "kernel": ["rbf", "linear"]},
    cv=5,
    return_train_score=False,
    n_iter=2,  # random 2
)
rs.fit(iris.data, iris.target)
pd.DataFrame(rs.cv_results_)[["param_C", "param_kernel", "mean_test_score"]]

,param_C,param_kernel,mean_test_score
0,20,rbf,0.966667
1,1,rbf,0.980000


In [151]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# json format
model_params = {
    "svm": {
        "model": svm.SVC(gamma="auto"),
        "params": {"C": [1, 10, 20], "kernel": ["rbf", "linear"]},
    },
    "random_forest": {
        "model": RandomForestClassifier(),
        "params": {"n_estimators": [1, 5, 10]},
    },
    "logistic_regression": {
        "model": LogisticRegression(solver="liblinear", multi_class="auto"),
        "params": {"C": [1, 5, 10]},
    },
}

In [152]:
# deprecated stuff
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

scores = []

for model_name, mp in model_params.items():
    clf = GridSearchCV(mp["model"], mp["params"], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append(
        {
            "model": model_name,
            "best_score": clf.best_score_,
            "best_params": clf.best_params_,
        }
    )

df = pd.DataFrame(scores, columns=["model", "best_score", "best_params"])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.953333,{'n_estimators': 1}
2,logistic_regression,0.966667,{'C': 5}
